<a href="https://colab.research.google.com/github/jittapont/bank_proof_classification/blob/master/Image_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [0]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [8]:
import pickle
import os
import matplotlib.pyplot as plt
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from keras.utils.np_utils import to_categorical
import datetime
%tensorflow_version 1.15.0

`%tensorflow_version` only switches the major version: `1.x` or `2.x`.
You set: `1.15.0`. This will be interpreted as: `1.x`.


TensorFlow is already loaded. Please restart the runtime to change versions.


In [0]:
with open("/content/drive/My Drive/Dataset/X.pickle","rb") as pickle_in:
    X = pickle.load(pickle_in)

In [0]:
with open("/content/drive/My Drive/Dataset/y.pickle","rb") as pickle_in:
    y = pickle.load(pickle_in)

In [12]:
X.shape

(44000, 100, 100, 1)

In [0]:
y = to_categorical(y)

In [0]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize

import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn't guaranteed
gpu = GPUs[0]

def printm():
  process = psutil.Process(os.getpid())
  print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " I Proc size: " + humanize.naturalsize( process.memory_info().rss))
  print('GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB'.format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))

printm()

  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 9.1 GB  I Proc size: 5.0 GB
GPU RAM Free: 11370MB | Used: 71MB | Util   1% | Total 11441MB


In [18]:
model = Sequential()

model.add(Conv2D(256, (3, 3), input_shape=X.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(64))
model.add(Dense(64))

model.add(Dense(y.shape[1]))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X, y, batch_size=128, epochs=30, validation_split=0.3)

Train on 30799 samples, validate on 13201 samples
Epoch 1/30
30799/30799 [==============================] - 39s 1ms/sample - loss: 0.1703 - acc: 0.9506 - val_loss: 0.0190 - val_acc: 0.9964
Epoch 2/30
30799/30799 [==============================] - 38s 1ms/sample - loss: 0.0171 - acc: 0.9963 - val_loss: 0.0129 - val_acc: 0.9973
Epoch 3/30
30799/30799 [==============================] - 38s 1ms/sample - loss: 0.0068 - acc: 0.9984 - val_loss: 0.0095 - val_acc: 0.9987
Epoch 4/30
30799/30799 [==============================] - 38s 1ms/sample - loss: 0.0018 - acc: 0.9996 - val_loss: 0.0088 - val_acc: 0.9985
Epoch 5/30
30799/30799 [==============================] - 38s 1ms/sample - loss: 0.0029 - acc: 0.9993 - val_loss: 0.0109 - val_acc: 0.9985
Epoch 6/30
30799/30799 [==============================] - 38s 1ms/sample - loss: 0.0013 - acc: 0.9996 - val_loss: 0.0141 - val_acc: 0.9983
Epoch 7/30
30799/30799 [==============================] - 38s 1ms/sample - loss: 0.0026 - acc: 0.9993 - val_loss: 0.

In [0]:
model_path = r'/content/drive/My Drive/OCR scripts'
model.save(os.path.join(
    model_path,
    f"image_classification_model_{datetime.datetime.now().strftime('%Y-%m-%d')}.model")
)

In [0]:
model.save(
    f"image_classification_model_{datetime.datetime.now().strftime('%Y-%m-%d')}.model")

In [0]:
import cv2
import tensorflow as tf
import numpy as np
categories = ['BAY', 'BBL', 'GSB', 'KBANK', 'KTB', 'Others', 'SCB', 'TBANK', 'TMB']

def prepare(filepath):
    IMG_SIZE = 100
    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE)) 
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 1) 

model = tf.keras.models.load_model("/content/Image_Classification.model")
prediction = model.predict([prepare('/content/678941063_14863738349228379046.jpeg')])
print(prediction[0])
print(categories[np.argmax(prediction[0])])

[0. 0. 0. 1. 0. 0. 0. 0. 0.]
KBANK


In [0]:
prediction[0]

array([0., 0., 0., 1., 0., 0., 0., 0., 0.], dtype=float32)